In [59]:
import numpy as np
import random

In [60]:
group_size = 3

students = [
    {'id':1, 'name':'alexandra','gender':'f', 'location':'online', 'language':'german'},
    {'id':2, 'name':'josie','gender':'f', 'location':'onsite', 'language':'english'},
    {'id':3, 'name':'laurel','gender':'f', 'location':'online', 'language':'both'},
    {'id':4, 'name':'bob','gender':'m', 'location':'onsite', 'language':'german'},
    {'id':5, 'name':'justin','gender':'m', 'location':'online', 'language':'german'},
    {'id':6, 'name':'aaron','gender':'m', 'location':'online', 'language':'english'},
    {'id':7, 'name':'frances','gender':'f', 'location':'onsite', 'language':'english'},
    {'id':8, 'name':'sally','gender':'f', 'location':'onsite', 'language':'both'},
    {'id':9, 'name':'claire','gender':'f', 'location':'online', 'language':'english'},
    {'id':10, 'name':'carol','gender':'f', 'location':'onsite', 'language':'both'},
    {'id':11, 'name':'lauren','gender':'f', 'location':'online', 'language':'german'},
    {'id':12, 'name':'nora','gender':'f', 'location':'onsite', 'language':'english'},
    {'id':13, 'name':'charlotte','gender':'f', 'location':'online', 'language':'both'},
    {'id':14, 'name':'camille','gender':'f', 'location':'onsite', 'language':'both'},
    {'id':15, 'name':'ben','gender':'m', 'location':'onsite', 'language':'german'},           
    {'id':16, 'name':'dan','gender':'m', 'location':'online', 'language':'english'},
    {'id':17, 'name':'harold','gender':'m', 'location':'online', 'language':'english'},
    {'id':18, 'name':'evelyn','gender':'f', 'location':'online', 'language':'both'},
    {'id':19, 'name':'leigh-ann','gender':'f', 'location':'onsite', 'language':'both'},
    {'id':20, 'name':'melissa','gender':'f', 'location':'online', 'language':'german'},  
    {'id':21, 'name':'brian','gender':'m', 'location':'onsite', 'language':'english'},
    ]

Random population

In [61]:
def generate_population(students, groupsize):

    students_copy = students.copy()

    arr = np.zeros(shape=(int(len(students_copy)/group_size), group_size, 1), dtype=int)

    for i in range(0,len(arr)):
        for j in range(0,groupsize):
            x = random.choice(students_copy)
            arr[i][j] = x['id']
            students_copy.remove(x)
    return arr

In [62]:
population = generate_population(students, group_size)

In [63]:
def show_population(array, students):
    for n,i in enumerate(array): 
        print('\nGroup:' ,n+1)
        for j in i:
            for k in students:
                if k['id'] == int(j[0]):
                    print(k['name'])

In [64]:
show_population(population, students)


Group: 1
bob
sally
nora

Group: 2
aaron
lauren
leigh-ann

Group: 3
dan
laurel
josie

Group: 4
alexandra
claire
melissa

Group: 5
charlotte
carol
brian

Group: 6
harold
ben
justin

Group: 7
camille
frances
evelyn


Evaluation: 

* Group has to be mixed with atleast one female and one male
* Group has to have the same language
* Group has to work in same location
* Group has to have the same language or people that speak both languages

In [65]:
def evaluate(array, students):

    total = []
    for i in array:
        local = 0
        gender = []
        location = []
        langauge = []
        for j in i: # Group
            for k in students:
                if k['id'] == int(j[0]):
                    
                    gender.append(k['gender'])
                    location.append(k['location'])
                    langauge.append(k['language'])

        if gender.count('m') <= 2 or gender.count('f') <= 2:
            local += 1

        if len(set(location)) == 1:
            local += 1

        if len(set(langauge)) == 1:
            local += 1

        total.append(local)

    return sum(total)

In [66]:
evaluate(population, students)

9

Selection and getting Parents

In [67]:
def get_parents(scores, total_population):

    parents = []

    max_score = max(scores)
    max_index = scores.index(max_score)
    parents.append(total_population[max_index])
    scores.remove(max_score)

    max_score = max(scores)
    max_index = scores.index(max_score)
    parents.append(total_population[max_index])
    scores.remove(max_score)

    return parents

In [68]:
dummy_populations = [generate_population(students, group_size) for i in range(0,10)]
dummy_scores = [evaluate(i, students) for i in dummy_populations]

In [69]:
a,b = get_parents(dummy_scores, dummy_populations)

Reproduction (and Mutation)

In [70]:
def crossover(parent1, parent2):

    if len(parent1) != len(parent2):
        print('Parents are not the same length')
        return
        
    child = []
    for i in range(0,len(parent1)):
        if i % 2 == 0:
            child.append(parent1[i])
        else:
            child.append(parent2[i])

    child = np.array(child)
    child = np.reshape(child, newshape=(len(parent1), 3, 1))
    return child

In [248]:
def mutation(array, rate):

    if random.uniform(0, 1) < rate:
        
        number1 = random.randint(1,len(array)*group_size)
        number2 = random.randint(1,len(array)*group_size)

        while number1 == number2:
            number2 = random.randint(1,len(array)*group_size)
        
        mutation_location1 = np.argwhere(array == [number1])
        mutation_location2 = np.argwhere(array == [number2])

        print(mutation_location1, mutation_location2)
        print('----')

        array[mutation_location2[0][0]][mutation_location2[0][1]] = number1
        array[mutation_location1[0][0]][mutation_location1[0][1]] = number2

    return array

In [250]:
def delete_lowest(array, scores):

    lowest_score = min(scores)
    lowest_index = scores.index(lowest_score)
    array.pop(lowest_index)
    scores.pop(lowest_index)

    return array, scores

The full algorithm

In [224]:
def genetic_algorithm(n_populations, students, group_size, mutation_rate):

    scores = []
    parents = []
    iterations = 0
    score = 0

    total_population = [generate_population(students, group_size) for _ in range(0,n_populations)]

    while score <= 21:

        scores = [evaluate(i, students) for i in total_population]
        parents = get_parents(scores, total_population)
        child = crossover(parents[0], parents[1])
        child = mutation(child, mutation_rate)
        total_population.append(child)
        scores.append(evaluate(child, students))
        total_population, scores = delete_lowest(total_population, scores)

        score = max(scores)
        iterations += 1
        print(f'Itertaion:{iterations} - score: {score}')

        # scores.clear()

In [249]:
genetic_algorithm(10, students, group_size, 0.1)

Itertaion:1 - score: 11
Itertaion:2 - score: 11
Itertaion:3 - score: 11
Itertaion:4 - score: 11
Itertaion:5 - score: 11
Itertaion:6 - score: 11
[[3 2 0]] [[0 1 0]]
----
Itertaion:7 - score: 12
Itertaion:8 - score: 12
Itertaion:9 - score: 12
Itertaion:10 - score: 12
Itertaion:11 - score: 12
Itertaion:12 - score: 12
Itertaion:13 - score: 12
Itertaion:14 - score: 12
Itertaion:15 - score: 12
Itertaion:16 - score: 12
[[0 2 0]] [[6 2 0]]
----
Itertaion:17 - score: 12
Itertaion:18 - score: 12
Itertaion:19 - score: 12
[[1 1 0]] [[6 2 0]]
----
Itertaion:20 - score: 12
Itertaion:21 - score: 12
Itertaion:22 - score: 12
Itertaion:23 - score: 12
Itertaion:24 - score: 12
Itertaion:25 - score: 12
Itertaion:26 - score: 12
Itertaion:27 - score: 12
Itertaion:28 - score: 12
Itertaion:29 - score: 12
Itertaion:30 - score: 12
[[4 0 0]] [[1 0 0]]
----
Itertaion:31 - score: 12
Itertaion:32 - score: 12
Itertaion:33 - score: 12
Itertaion:34 - score: 12
Itertaion:35 - score: 12
Itertaion:36 - score: 12
[[0 0 0]]

KeyboardInterrupt: 